# Assignment - Segmenting and clustering neighborhoods in Toronto

-  Scrape code from the following [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
-  Obtain data that is in the list of postal codes 
-  Transform the data into a pandas dataframe

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [1]:
import pandas as pd
import numpy as np
#Beautiful Soup is a Python library for pulling data out of HTML and XML files.


#  'None' value means unlimited.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
#from urllib import urlopen

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df_Toronto = pd.DataFrame(columns = columns)
df_Toronto

,PostalCode,Borough,Neighborhood


In [4]:
url_postalcodes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
!wget -O 'toronto_postal_codes.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

print('Downloaded file')

with open('toronto_postal_codes.html') as postalcodes:
    soup = BeautifulSoup(postalcodes, 'html.parser')

#soup = BeautifulSoup(urlopen(url_postalcodes), "html5lib")

--2018-12-11 00:01:02--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80297 (78K) [text/html]
Saving to: ‘toronto_postal_codes.html’

toronto_postal_code 100%[===================>]  78.42K  --.-KB/s    in 0.05s   

2018-12-11 00:01:02 (1.48 MB/s) - ‘toronto_postal_codes.html’ saved [80297/80297]

Downloaded file


Beautiful Soup - 

Reference code for parsing td and tr tags
[https://www.airpair.com/python/posts/using-python-and-qgis-for-geospatial-visualization#rmxtgga1m5PJ9Vp8.30](Using Python and QGIS for geospatial visualizations - a Case Study)

In [6]:
soup.prettify()
data = []
#print(df_Toronto)
if ('wikitable' in soup.table['class']):
    for row in soup.table.tbody('tr'):
        if row('th'):
            continue  # header row
        row_data = row('td')
        if (row_data[1].text == 'Not assigned'):
            continue
         
        df_Toronto = df_Toronto.append({'PostalCode': row_data[0].text,
                                        'Borough': row_data[1].text,
                                        'Neighborhood': row_data[2].text}, ignore_index=True)
        
df_Toronto
df_Toronto.shape


(212, 3)

In [7]:
df_Toronto.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [8]:
df_Toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Good [tutorial](https://www.youtube.com/watch?v=Wb2Tp35dZ-I) on pandas groupby - split, apply, combine

Perform a computation on the grouped data
For each group combine the neighborhood column values (Aggregation)

In [9]:
df_Toronto['Neighborhood'] = df_Toronto['Neighborhood'].map(lambda x:x.rstrip("\n"))

In [10]:
df_Toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Groupby is applied on columns 'PostalCode' and 'Borough'. With agg function you can use different functions on different columns. Concatenate the elements in column 'Neighborhood' while inserting a ', ' between the words 

The result of the aggregation will have the group names as the new index along the grouped axis. In the case of multiple keys, the result is a MultiIndex by default, though this can be changed by using the as_index option:

In [13]:
df_Toronto = df_Toronto.groupby(['PostalCode', 'Borough'], as_index = False).agg({'Neighborhood' : ', '.join})

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

Returns a pd.Series object consisting of the 'Neighborhood' column for all rows where 'Neighborhood' is 'Not assigned'
We assign df_Toronto['Borough'] for all elements in column 'Neighborhood' where 'Neighborhood' was 'Not assigned'

In [17]:
df_Toronto.loc[df_Toronto['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_Toronto['Borough']

In [19]:
df_Toronto.shape

(103, 3)